# StructureData class(es)

In `aiida-atomistic` we provide two StructureData classes: 

1. `StructureData`, the AiiDA data type used to store a structure in the AiiDA database. It is ***immutable***, i.e. after the initialization we cannot modify it. It is just a container for structure information;
2. `StructureDataMutable`, a python class used to manipulate a structure object before to transform it into the immutable `StructureData`;
    
With respect to `orm.StructureData`, here we provide additional properties to be attached to the structure, e.g. *charges* and *magmoms*. Kind based definition of the sites is dropped, and the *kind_name* is attached to each site as a property. 
The two `StructureData` and `StructureDataMutable` share the same data structure; the difference is that the latter can be modified by the user after its initialization and, at variance with the `StructureData`, no strict validation checks are performed. Moreover, `StructureDataMutable` will have additional setter methods. 
The properties are stored under the `properties` attribute of the structure, which is a *pydantic* [`BaseModel`](https://docs.pydantic.dev/latest/api/base_model/) subclass, particularly convenient for data validation.

:::{note}
The `StructureDataMutable` is a pure python class, i.e. it is **not** and AiiDA node class. This means that it cannot be used as direct input of and AiiDA process. It must always be converted into a `StructureData` object before setting the input structure of the process (check the corresponding [section](#passing-from-structuredata-to-structuredatamutable-and-viceversa)).
:::

## `StructureData`(s) initialization

As both `StructureData` and `StructureDataMutable` share the same data structure, they also share the same inputs for the constructor: a set of keyword arguments describing the properties. These can be easily organized in a python dictionary. This dictionary and the data stored in the AiiDA database have a 1-to1 correspondence:

In [1]:
from aiida import load_profile
load_profile()

from aiida_atomistic import StructureData, StructureDataMutable

structure_dict = {
    'pbc': [True, True, True],
    'cell': [[2.75, 2.75, 0.0], [0.0, 2.75, 2.75], [2.75, 0.0, 2.75]],
    'symbols': ['Si', 'Si'],
    'kinds': ['Si', 'Si'],
    'positions': [
        [0.0, 0.0, 0.0],
        [3.84, 1.3576450198781713, 1.9200000000000006]
    ],
}

mutable_structure = StructureDataMutable(**structure_dict)
structure = StructureData(**structure_dict)

print("Immutable StructureData:\n")
print("Immutable cell: ",structure.properties.cell)
print("Immutable pbc: ",structure.properties.pbc)
print("Immutable sites: ",structure.properties.sites)
print("First immutable site: ",structure.properties.sites[0].dict())

print("\nMutable StructureDataMutable:\n")
print("Mutable cell: ",mutable_structure.properties.cell)
print("Mutable pbc: ",mutable_structure.properties.pbc)
print("Mutable sites: ",mutable_structure.properties.sites)
print("First mutable site: ",mutable_structure.properties.sites[0].dict())

Immutable StructureData:

Immutable cell:  [[2.75, 2.75, 0.0], [0.0, 2.75, 2.75], [2.75, 0.0, 2.75]]
Immutable pbc:  [True, True, True]
Immutable sites:  [<SiteImmutable: kind name 'Si' @ 0.0,0.0,0.0>, <SiteImmutable: kind name 'Si' @ 3.84,1.3576450198781713,1.9200000000000006>]
First immutable site:  {'symbols': 'Si', 'kinds': 'Si', 'positions': [0.0, 0.0, 0.0], 'masses': 28.0855, 'charges': 0.0, 'magmoms': [0.0, 0.0, 0.0], 'weights': (1.0,)}

Mutable StructureDataMutable:

Mutable cell:  [[2.75, 2.75, 0.0], [0.0, 2.75, 2.75], [2.75, 0.0, 2.75]]
Mutable pbc:  [True, True, True]
Mutable sites:  [<SiteImmutable: kind name 'Si' @ 0.0,0.0,0.0>, <SiteImmutable: kind name 'Si' @ 3.84,1.3576450198781713,1.9200000000000006>]
First mutable site:  {'symbols': 'Si', 'kinds': 'Si', 'positions': [0.0, 0.0, 0.0], 'masses': 28.0855, 'charges': 0.0, 'magmoms': [0.0, 0.0, 0.0], 'weights': (1.0,)}


As we provide the `structure_dict` to the constructor of our two structure data classes, it is immediately used to build up the `properties` attribute. Each site is stored as `SiteMutable` (`SiteImmutable`) object for the mutable (immutable) case. Mutability (immutability) is inherited from the corresponding StructureData class used.

The structure can be dumped into a dictionary using the `to_dict` method of the structure, which is nothing else than a wrapper for the *pydantic* *BaseModel* `model_dump` method of the *properties* attribute:

In [2]:
structure.to_dict()

{'pbc': [True, True, True],
 'cell': [[2.75, 2.75, 0.0], [0.0, 2.75, 2.75], [2.75, 0.0, 2.75]],
 'custom': None,
 'symbols': ['Si', 'Si'],
 'positions': [[0.0, 0.0, 0.0],
  [3.84, 1.3576450198781713, 1.9200000000000006]],
 'kinds': ['Si', 'Si'],
 'weights': [(1.0,), (1.0,)],
 'masses': [28.0855, 28.0855],
 'charges': [0.0, 0.0],
 'magmoms': [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]],
 'hubbard': {'parameters': [],
  'projectors': 'ortho-atomic',
  'formulation': 'dudarev'},
 'cell_volume': 41.59375,
 'dimensionality': {'dim': 3, 'label': 'volume', 'value': 41.59375},
 'sites': [{'symbols': 'Si',
   'kinds': 'Si',
   'positions': [0.0, 0.0, 0.0],
   'masses': 28.0855,
   'charges': 0.0,
   'magmoms': [0.0, 0.0, 0.0],
   'weights': (1.0,)},
  {'symbols': 'Si',
   'kinds': 'Si',
   'positions': [3.84, 1.3576450198781713, 1.9200000000000006],
   'masses': 28.0855,
   'charges': 0.0,
   'magmoms': [0.0, 0.0, 0.0],
   'weights': (1.0,)}],
 'formula': 'Si2'}

:::{important}

Some properties are generated automatically, like *sites*, *formula*,*cell_volume* and so on, and some other properties are set by default if not provided, e.g. the *kinds* and *weights* of each site. 
It crucial to understand that we can never set by hands the `computed` properties, not even in the `StructureDataMutable`: these are derived properties. 

The set of computed properties can be printed easily using the `model_computed_fields` attribute of the `properties` model:

In [3]:
structure.properties.model_computed_fields.keys()

dict_keys(['cell_volume', 'dimensionality', 'sites', 'formula'])

Useful methods are provided to help the user modify the structure. For example, we can modify a site just using the `update_site` method of the structure class:

In [4]:
import numpy as np

print("Initial charge on the first site: ", mutable_structure.properties.sites[0].charges)

mutable_structure.update_site(
    site_index=0, 
    charges=+1)

print("Updated charge on the first site: ", mutable_structure.properties.sites[0].charges)

Initial charge on the first site:  0.0
Updated charge on the first site:  1.0


To visualize the *supported* properties, use the `get_supported_properties` method of the structure class.
To visualize the *defined* properties for the structure, you can use the `get_defined_properties`.
:::

we can also provide the dictionary containing the sites as list of dictionaries, each of them describing a site. This is useful when we want to provide additional information for each site, such as the occupancy or the magnetic moment. In this case, the dictionary should have the following structure:
    
```python
structure_dict = {
    'cell':[[2.75,2.75,0],[0,2.75,2.75],[2.75,0,2.75]],
    'pbc': [True,True,True],
    'sites':[
        {
            'symbols':'Si',
            'positions':[3/4, 3/4, 3/4],
        },
        {
            'symbols':'Si',
            'positions':[1/2, 1/2, 1/2],
        },
    ],
}
```


### Initialization from ASE or Pymatgen

If we already have an [ASE](https://wiki.fysik.dtu.dk/ase/) `Atoms` or a [Pymatgen](https://pymatgen.org/index.html) `Structure` object, we can initialize our `StructureData` by means of the built-in `from_ase` and `from_pymatgen` methods.

For ASE:

In [5]:
from ase.build import bulk
atoms = bulk('Cu', 'fcc', a=3.6)
atoms.set_initial_charges([1,])

mutable_structure = StructureDataMutable.from_ase(atoms)
structure = StructureData.from_ase(atoms)

structure.to_dict()

{'pbc': [True, True, True],
 'cell': [[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]],
 'custom': None,
 'symbols': ['Cu'],
 'positions': [[0.0, 0.0, 0.0]],
 'kinds': ['Cu0'],
 'weights': [(1.0,)],
 'masses': [63.546],
 'charges': [1.0],
 'magmoms': [[0.0, 0.0, 0.0]],
 'hubbard': {'parameters': [],
  'projectors': 'ortho-atomic',
  'formulation': 'dudarev'},
 'cell_volume': 11.664000000000001,
 'dimensionality': {'dim': 3, 'label': 'volume', 'value': 11.664000000000001},
 'sites': [{'symbols': 'Cu',
   'kinds': 'Cu0',
   'positions': [0.0, 0.0, 0.0],
   'masses': 63.546,
   'charges': 1.0,
   'magmoms': [0.0, 0.0, 0.0],
   'weights': (1.0,)}],
 'formula': 'Cu'}

In the Pymatgen case:

In [6]:
from pymatgen.core import Lattice, Structure, Molecule

coords = [[0, 0, 0], [0.75,0.5,0.75]]
lattice = Lattice.from_parameters(a=3.84, b=3.84, c=3.84, alpha=120,
                                beta=90, gamma=60)
struct = Structure(lattice, ["Si", "Si"], coords)

struct.sites[0].properties["charge"]=1

mutable_structure = StructureDataMutable.from_pymatgen(struct)
structure = StructureData.from_pymatgen(struct)

mutable_structure.properties.charges

[1.0, 0.0]

Moreover, we also provide `to_ase` and `to_pymatgen` methods to obtain the corresponding instances.

:::{note}
When a StructureData is initialized from ASE or Pymatgen, kinds are automatically detected. For further details, please see the section on [Automatic kinds generation](#automatic-kinds-generation). 
:::

## Mutation of a `StructureDataMutable` instance

Let's suppose you want to update some property in the `StructureData` before to use it in a calculation. You cannot. The way to go is either to use ASE or Pymatgen to modify your object and store it back into `StructureData`, or to use the `StructureDataMutable` and its mutation methods, and then convert it into `StructureData`.
The latter method is the preferred one, as you then have support also for additional properties like the Hubbard U, is not supported in ASE and Pymatgen.

### Conversion from StructureData to StructureDataMutable and viceversa 

We provide two methods to convert back and forth between `StructureData` and `StructureDataMutable`. This is useful if we start with a `StructureData` and we need to modify it:

In [7]:
mutable_structure.to_immutable() # returns an instance of StructureData
structure.to_mutable() # returns an instance of StructureDataMutable

### Methods to update properties

`StructureDataMutable` properties can be modified directly, but also the class contains several setter (`set_*`) methods and more, needed to update a structure. Let's suppose we start from an immutable `StructureData` and we want to update the charges (and the corresponding kinds):

In [8]:
mutable_structure = structure.to_mutable()

print(f"Initial charges, kinds:\n{mutable_structure.properties.charges}, {mutable_structure.properties.kinds}")

mutable_structure.set_charges([-1, +1])
mutable_structure.set_kinds(['Si1','Si2'])

new_structure = mutable_structure.to_immutable()

print(f"New charges, kinds:\n{new_structure.properties.charges}, {new_structure.properties.kinds}")

Initial charges, kinds:
[1.0, 0.0], ['Si0', 'Si1']
New charges, kinds:
[-1.0, 1.0], ['Si1', 'Si2']


:::{note} Preserving the provenance
When starting from a `StructureData`, passing to a `StructureDataMutable` and then generating a new modified `StructureData`, we lose provenance. To keep it, we should do the modification by means of an AiiDA [*calcfunction*](https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/calculations/concepts.html#calculation-functions), which takes as input (output) the starting (modified) `StructureData`. 
For example, we provide the `generate_striped_structure` utility `calcfunction` to obtain a new `StructureData` without a given property:

```python
from aiida import orm
from aiida_atomistic.data.structure.utils import generate_striped_structure

new_structure = generate_striped_structure(structure, orm.List(["charge"])) # we reset the charges to the default value (zero charge).
```
:::

It is also possible to `add_atom`, `pop_atom`, `update_site` and so on.
Indeed, we can start from and empty `StructureDataMutable` (i.e., from scratch):

In [9]:
mutable_structure = StructureDataMutable()

print("Initial mutable structure is empty:\n", mutable_structure.to_dict()["sites"])

mutable_structure.set_cell([[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])
mutable_structure.add_atom(**{
            'symbols':'Si',
            'positions':[3/4, 3/4, 3/4],
            'charges': 1,
            'kinds': 'Si1'
        })

mutable_structure.add_atom(
            symbols = 'Si',
            positions = [1/2, 1/2, 1/2],
            charges = 0,
            kinds = 'Si2'
        )

print("\nModified structure is:")
mutable_structure.to_dict()["sites"]

Initial mutable structure is empty:
 []

Modified structure is:


[{'symbols': 'Si',
  'kinds': 'Si1',
  'positions': [0.75, 0.75, 0.75],
  'masses': 28.0855,
  'charges': 1.0,
  'magmoms': [0.0, 0.0, 0.0],
  'weights': (1.0,)},
 {'symbols': 'Si',
  'kinds': 'Si2',
  'positions': [0.5, 0.5, 0.5],
  'masses': 28.0855,
  'charges': 0.0,
  'magmoms': [0.0, 0.0, 0.0],
  'weights': (1.0,)}]

:::{note} Set a property for a given kind
It is possible to set properties for a given `kinds` in the structure. This allows the user to avoid to loop over all the sites to update their properties only if the site is of a given kind:

```python
mutable_structure.set_kind_property(name="magmoms", kinds="Si1", value=[0,0,-1])
mutable_structure.properties.magmoms
```

:::

To remove properties, we can use the `clear_property` method, providing as input parameter the string representing the name of the property: `mutable_structure.clear_property("magmoms")`

###  Slicing a structure

It is possible to *slice* a structure, i.e. returning only a part of it (in terms of sites). The method returns a new sliced `StructureDataMutable` (`StructureData`) instance.

In [10]:
sliced_structure = mutable_structure[:1]
sliced_structure.to_dict()

{'pbc': [True, True, True],
 'cell': [[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]],
 'custom': None,
 'symbols': ['Si'],
 'positions': [[0.75, 0.75, 0.75]],
 'kinds': ['Si1'],
 'weights': [(1.0,)],
 'masses': [28.0855],
 'charges': [1.0],
 'magmoms': [[0.0, 0.0, 0.0]],
 'hubbard': {'parameters': [],
  'projectors': 'ortho-atomic',
  'formulation': 'dudarev'},
 'cell_volume': 11.664000000000001,
 'dimensionality': {'dim': 3, 'label': 'volume', 'value': 11.664000000000001},
 'sites': [{'symbols': 'Si',
   'kinds': 'Si1',
   'positions': [0.75, 0.75, 0.75],
   'masses': 28.0855,
   'charges': 1.0,
   'magmoms': [0.0, 0.0, 0.0],
   'weights': (1.0,)}],
 'formula': 'Si'}

## Automatic kinds generation

It is possible to automatically detect kinds for a given structure. This is done by default when initializing the structure from ASE or Pymatgen. Moreover, the kind can be also generated during the `to_dict` call, such that our output_dictionary will already have the detected kinds. This is valid also for the `to_mutable`, `to_immutable` methods. 
In summary, we can control automatic kind detection in the initialization of our StructureData/StructureDataMutable:

1. new_structuredata = StructureData.from_ase(ase_structure, detect_kinds=False) # default is True
2. new_structuredata = StructureData.from_pymatgen(pymatgen_structure, detect_kinds=False) # default is True
3. new_structuredata = StructureData(**old_structuredata.to_dict(detect_kinds=True)) # default is False
4. new_structuredata = mutable_structure.to_immutable(detect_kinds=True) # default is False

In [11]:
Fe_BCC_dictionary = {
        'pbc': (True, True, True),
        'cell': [
                [2.8403, 0.0, 1.7391821518091137e-16],
                [-1.7391821518091137e-16, 2.8403, 1.7391821518091137e-16],
                [0.0, 0.0, 2.8403]
        ],
        'sites': [
                {
                        'symbols': 'Fe',
                        'masses': 55.845,
                        'positions': [0.0, 0.0, 0.0],
                        'charges': 0.0,
                        'magmoms': [2.5, 0.1, 0.1],
                        'kinds': 'Fe'
                },
                {
                        'symbols': 'Fe',
                        'masses': 55.845,
                        'positions': [1.42015, 1.42015, 1.4201500000000002],
                        'charges': 0.0,
                        'magmoms': [2.4, 0.1, 0.1],
                        'kinds': 'Fe'
                }
        ]
}

mutable_structure = StructureDataMutable(**Fe_BCC_dictionary)

new_mutable_structure = StructureDataMutable(**mutable_structure.to_dict(detect_kinds=True))
new_mutable_structure.to_dict()

mutable_structure = StructureDataMutable(**Fe_BCC_dictionary)

new_mutable_structure = StructureDataMutable(**mutable_structure.to_dict(detect_kinds=True))
new_mutable_structure.to_dict()

{'pbc': [True, True, True],
 'cell': [[2.8403, 0.0, 1.7391821518091137e-16],
  [-1.7391821518091137e-16, 2.8403, 1.7391821518091137e-16],
  [0.0, 0.0, 2.8403]],
 'custom': None,
 'symbols': ['Fe', 'Fe'],
 'positions': [[0.0, 0.0, 0.0], [1.42015, 1.42015, 1.4201500000000002]],
 'kinds': ['Fe0', 'Fe1'],
 'weights': [(1.0,), (1.0,)],
 'masses': [55.845, 55.845],
 'charges': [0.0, 0.0],
 'magmoms': [[2.5, 0.1, 0.1], [2.4, 0.1, 0.1]],
 'hubbard': {'parameters': [],
  'projectors': 'ortho-atomic',
  'formulation': 'dudarev'},
 'cell_volume': 22.913563806827,
 'dimensionality': {'dim': 3, 'label': 'volume', 'value': 22.913563806827},
 'sites': [{'symbols': 'Fe',
   'kinds': 'Fe0',
   'positions': [0.0, 0.0, 0.0],
   'masses': 55.845,
   'charges': 0.0,
   'magmoms': [2.5, 0.1, 0.1],
   'weights': (1.0,)},
  {'symbols': 'Fe',
   'kinds': 'Fe1',
   'positions': [1.42015, 1.42015, 1.4201500000000002],
   'masses': 55.845,
   'charges': 0.0,
   'magmoms': [2.4, 0.1, 0.1],
   'weights': (1.0,)}],


To return the list of sites with the automatically generated kinds, you can also invoke the `get_kinds` method in this way

In [12]:
sites_with_kinds = mutable_structure.get_kinds(ready_to_use=True)
sites_with_kinds

[{'kinds': 'Fe0',
  'masses': 55.845,
  'charges': 0.0,
  'magmoms': [2.5, 0.1, 0.1],
  'symbols': 'Fe',
  'positions': [0.0, 0.0, 0.0]},
 {'kinds': 'Fe1',
  'masses': 55.845,
  'charges': 0.0,
  'magmoms': [2.4, 0.1, 0.1],
  'symbols': 'Fe',
  'positions': [1.42015, 1.42015, 1.4201500000000002]}]

## How to Query StructureData using properties

Thanks to the additional computed properties in our `StructureData` (*formula*, *symbols*, *kinds*, *masses*, *charges*, *magmoms*, *positions*, *cell_volume*, *dimensionality*), we can easily query for a structure:

In [13]:
from aiida.orm import QueryBuilder

stored = new_mutable_structure.to_immutable().store()
print(stored.pk)

qb = QueryBuilder()
qb.append(StructureData, 
          filters={'attributes.formula': 'Fe2'},
          )

print(qb.all()[0])

2031
[<StructureData: uuid: 8851c51a-0560-404b-abba-82359ee5a595 (pk: 2029)>]


## How to define alloys and deal with vacancies

It is possible to define more than one element for a given site, i.e. to define an *alloy*. This can be done by providing as symbol the combination of the symbols, and also the corresponding *weights* tuple:

In [14]:
structure = StructureDataMutable(**{
  'pbc': [True, True, True],
  'cell': [[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]],
  'sites': [{
    'symbols': 'CuAl',  # or, alternatively, ['Cu', 'Al']
    'positions': [0.0, 0.0, 0.0],
    'weights': (0.5, 0.5)
  }],
})

structure.properties.sites[0].dict()

{'symbols': 'CuAl',
 'kinds': 'CuAl',
 'positions': [0.0, 0.0, 0.0],
 'masses': 45.263768999999996,
 'charges': 0.0,
 'magmoms': [0.0, 0.0, 0.0],
 'weights': (0.5, 0.5)}

if not provided, the mass is computed accordingly to the symbols and weights. Vacancies are detected when the sum of the weights is less than 1.

In [15]:
print(structure.is_alloy)
print(structure.has_vacancies)

True
False


## How to add custom properties

It is possible to add custom properties at the `StructureData` level (not at the `Site` level). These are defined as properties which are not officially supported in the `atomistic` package.
To do that, it is sufficient to put the corresponding property under the `custom` Field, a dictionary which should contain the custom property names as keys, followed by the corresponding value:

In [ ]:
structure = StructureData(**{
  'pbc': [True, True, True],
  'cell': [[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]],
  'sites': [{
    'symbols': 'Cu',
    'positions': [0.0, 0.0, 0.0],
  }],
  'custom': {
    'electronic_type': 'metal',
  }
})

structure.properties.custom

:::{note}
:class: dropdown
Automatic serialization of the custom properties is done when the model is dumped (e.g. when the structure is stored in the AiiDA database). If serialization is not possible, an error is retrieved.

The defined property should be used in the plugin as any other property: if you just want to store additional information on the structure, which do not need to be used in a calculation, store it in the `extras`:

```python
structure.base.extras.set('electronic_type', 'metal')
```

:::

## Backward compatibility support

As minimal backward compatibility (which will be dropped in future versions), we implemented the `to_atomistic` method in the old `orm.StructureData`, so that it is possible to start from an old structure and dump the corresponding instance of the new data type.